In [1]:
DATALEN = 100#12#4000#000

import utils_lstm as utils_lstm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os,sys,inspect,pickle,json,time,datetime,re; root = os.path.dirname(os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))))
import random as rn
import utils
%matplotlib inline

conf = json.load(open('config_zoo_play.json')) #conf = json.loads(config_lgb.replace("'", "\""))
TARGET = 'activity'
dates = pd.to_datetime(pd.date_range('2018-02-13', periods=DATALEN, freq='MIN', name='date'), errors='coerce') # freq='T'
# col1 = np.array([np.sin(i/(DATALEN/40)) for i in range(1,(len(dates)+1))]) * 3 # np.random.randn(len(dates))
# col2 = np.arange(len(dates)) * 1 # np.random.randn(len(dates))
# col3 = np.array([np.sin(i/(DATALEN/8)) for i in range(1,(len(dates)+1))]) * 2 # np.random.randn(len(dates))
# col4 = np.array([np.sin(i/(DATALEN/40)) for i in range(1,(len(dates)+1))]) * 2 # np.random.randn(len(dates))
# train_data_orig = pd.DataFrame({'activity':col1,'col2':col2,'col3':col3,'col4':col4}, index=dates)
col1 = np.arange(len(dates)) * 1
col2 = -np.arange(len(dates))#np.array([3 for i in range(len(dates))])
col3 = np.array([3 for i in range(len(dates))])

train_data_orig = pd.DataFrame({'activity':col1,'col2':col2,'col3':col3}, index=dates)

features_orig = train_data_orig.columns.tolist(); features_orig.remove(TARGET)
train_data = train_data_orig#train_data_orig[FEATURES + [TARGET]] # most correlated with activity
test_data = train_data_orig#test_data_orig[FEATURES] # most correlated with activity
#test_data = test_data_orig # use it when features=Target

FEATURES = features_orig

if conf['datasize']=='min':
    train = train_data[:10000]
    val   = train_data[-10000:]
elif conf['datasize']=='med':
    #train = train_data[:"2018-02-13 04:01:00"]#[:"2018-10-13 00:01:00"]
    #val   = train_data["2018-02-13 04:01:00":]#["2018-10-14 03:01:00":]
    train = train_data[:DATALEN//2]
    val   = train_data[DATALEN//2:]
    #train = train_data[:DATALEN/2]
    #val   = train_data[-DATALEN/2:]
elif conf['datasize']=='max':
    train = train_data
    val   = train_data
test  = test_data[FEATURES]

x_train, x_val, x_test, y_train, y_val = utils.split_cols_np(train, val, test, TARGET) ;print('x_train',x_train.shape, 'y_train',y_train.shape, 'x_val',x_val.shape, 'y_val',y_val.shape, 'x_test',x_test.shape)
x_train, x_val, x_test, y_train, y_val = x_train.astype('float64'), x_val.astype('float64'), x_test.astype('float64'), y_train.astype('float64'), y_val.astype('float64')
x_train_n, x_val_n, x_test_n, y_train_n, y_val_n = utils.normalize_by_train(x_train, x_val, x_test, y_train, y_val, train, TARGET) ;print('x_train_n',x_train_n.shape, 'y_train_n',y_train_n.shape, 'x_val_n',x_val_n.shape, 'y_val_n', y_val_n.shape)
#x_train_n, x_val_n, x_test_n, y_train_n, y_val_n = x_train, x_val, x_test, y_train, y_val

def evaluate(y_train_pred_n, y_val_pred_n, y_test_pred_n, conf_i, submit=False):
    # denormalize
    y_train_pred, y_val_pred, y_test_pred = utils.denormalize_by_train(y_train_pred_n, y_val_pred_n, y_test_pred_n, train, TARGET)

    # convert to pandas
    y_train_pred_p = pd.Series(y_train_pred.flatten(), index=train.index, name=TARGET+"_pred").sort_index()
    y_val_pred_p   = pd.Series(y_val_pred.flatten(),   index=val.index,   name=TARGET+"_pred").sort_index()
    if submit: y_test_pred_p  = pd.DataFrame(y_test_pred, columns=[TARGET], index=extra_data1.index) # activity_test_target

    # evaluate
    def mape(y_true, y_pred): return 100 * np.mean(np.abs((y_pred-y_true)/y_true))
    conf_i['score_train']= mape(train[TARGET].values, y_train_pred_p.values)
    conf_i['score_val']  = mape(val[TARGET].values,   y_val_pred_p.values)

    # submit
    curr_time = re.sub(r'^(.*)-(.*)-(.*) (.*):(.*):(.*)\.(.*)$',r'y\1_\2m_\3d_\4h_\5min_\6s_\7_',str(datetime.datetime.now()))
    if submit: y_test_pred_p.loc[test.index].to_csv(os.path.join('output',curr_time+mtype+'_sub.csv'))

    # add log to log_model.csv file
    log_file = 'log_'+mtype+'.csv'
    log_df = pd.DataFrame([conf_i], index=[curr_time])
    if log_file in os.listdir('output'): log_df = pd.read_csv(os.path.join('output', log_file), index_col=0).append(log_df)   
    log_df.to_csv(os.path.join('output', log_file))
    # add log to separate file
    #pd.DataFrame([conf_i], index=[curr_time]).to_csv(os.path.join('output',curr_time+mtype+'_log.csv'))
    return
#evaluated(y_train_pred_n, y_val_pred_n, y_test_pred_n, conf['lgb'], submit=True)

#train_data_orig.plot(subplots=True)


x_train (50, 2) y_train (50, 1) x_val (50, 2) y_val (50, 1) x_test (100, 2)
x_train_n (50, 2) y_train_n (50, 1) x_val_n (50, 2) y_val_n (50, 1)


C:\Users\user01\Desktop\sibur-production\model\playground\tf2_lstm_play\utils.py:20: RuntimeWarning: invalid value encountered in true_divide
  x_train_n = (x_train - center_x)/scale_x
C:\Users\user01\Desktop\sibur-production\model\playground\tf2_lstm_play\utils.py:21: RuntimeWarning: invalid value encountered in true_divide
  x_val_n   = (x_val   - center_x)/scale_x
C:\Users\user01\Desktop\sibur-production\model\playground\tf2_lstm_play\utils.py:22: RuntimeWarning: invalid value encountered in true_divide
  x_test_n  = (x_test  - center_x)/scale_x


In [2]:

for mtype in conf['models'].split(','):
    for conf_i in utils.split_model_config(conf[mtype]):
        if mtype=='tf':
            from train_tf import TrainTF
            model = TrainTF(conf_i)
        elif mtype=='lgb':
            from train_lgb import TrainLGB
            model = TrainLGB(conf_i)
        elif mtype=='mynn':
            from train_mynn import TrainMYNN
            model = TrainMYNN(conf_i)
        elif mtype=='tf_lstm':
            from train_tf_lstm_play import TrainTFLSTM
            model = TrainTFLSTM(conf_i)
        print('x_train_n',x_train_n.shape, 'y_train_n',y_train_n.shape, 'x_val_n',x_val_n.shape, 'y_val_n', y_val_n.shape)
        model.prepare_data(x_train_n, y_train_n, x_val_n, y_val_n)
        model.compile(x_train.shape)
        model.fit(x_train_n, y_train_n, x_val_n, y_val_n)
        y_train_pred_n = model.predict(x_train_n) 
        y_val_pred_n   = model.predict(x_val_n) 
        y_test_pred_n  = model.predict(x_test_n) 
#         y_train_pred_n = None
#         y_val_pred_n   = None
#         y_test_pred_n = None
        conf_i['history'] = model.get_costs() #model.history_callback
        conf_i['datasize'] = conf['datasize']
        conf_i['data'] = conf['data']
        conf_i['rs_global'] = conf['rs']
        conf_i['target'] = TARGET
        conf_i['features'] = FEATURES
        #conf_i['fegroup'] = fename
        evaluate(y_train_pred_n, y_val_pred_n, y_test_pred_n, conf_i, submit=False)
        #plot_costs(*model.get_costs())
    

x_train_n (50, 2) y_train_n (50, 1) x_val_n (50, 2) y_val_n (50, 1)
x_train_multi (44, 6, 3) y_train_multi (44, 9)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train for 2 steps, validate for 1 steps
1/2 [==============>...............] - ETA: 5s

UnknownError:  [_Derived_]  Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/StatefulPartitionedCall]] [Op:__inference_distributed_function_4672]

Function call stack:
distributed_function -> distributed_function -> distributed_function


In [ ]:
#for x, y in model.train_data_multi.take(1):
#print('x=',x.shape) # (16, 7, 2) (batch_size, past_history, features_count_with_target)
#print('y=',y.shape) # (16, 13) (batch_size, future_target)

In [ ]:
# for x, y in model.val_data_multi.take(1): # take one batch
#     utils_lstm.multi_step_plot(x[0], y[0], model.model.predict(x)[0], model.conf['step'])
#     print(model.model.predict(x)[0].shape)

In [ ]:
# for mtype in conf['models'].split(','):
#     utils.plot_loss(mtype,3)
# #utils.plot_loss('lgb')


In [ ]:
mtype = conf['models'].split(',')[0]
result = pd.read_csv('output/log_'+mtype+'.csv', index_col=0)[['score_train','score_val','target','datasize']]
result
# mtype = 'tf'
# pd.read_csv('output/log_'+mtype+'.csv', index_col=0)
# #pd.read_csv('output/log_'+mtype+'.csv', index_col=0)[['score_train','score_val']+conf[mtype]['tune'].split(',')].sort_values(by=['score_val'])